# Labsession Pyspark
In this labsession we will be covering pyspark, the pythong implementation of Apache Spark, which uses the MapReduce paradigm. As was the case for previous sessions, fill in the CODE_HERE placeholders.

The documentation for Pyspark is located at https://spark.apache.org/docs/latest/api/python/pyspark

In [19]:
import json
import re
from collections import Counter
#uncomment this when you installed pyspark yourself on a Windows computer following the given tutorial.
#import findspark
#findspark.init()
import numpy as np
import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
from pyspark.sql import functions as F

## First  let's rehearse some basic python concepts that we need in this lab session

### play around with the code to see how it behaves, use print to see what it's doing!

#### The lambda function and its equivalent normal definition

In [1]:
lambda_add_2 = lambda x: x+2

def add_2(x):
    return x+2

print(lambda_add_2(10))
print(add_2(10))

12
12


### How to access nested JSON keys and values

In [8]:
js_obj = {
    "list": [0,1,2,3],
    "nested_dict": {
        "key": "value"
    },
    "nested_list_of_dicts": [
        {
            "key": 0
        },
        {
            "key": 1
        }
    ]
}

print()
print(f"""js_obj["list"] = {js_obj["list"]}""")
print(f"""js_obj["list"][1] = {js_obj["list"][1]}""")
print(f"""js_obj["nested_dict"]["key"] = {js_obj["nested_dict"]["key"]}""")
print(f"""js_obj["nested_list_of_dicts"][1]["key"] = {js_obj["nested_list_of_dicts"][1]["key"]}""")


js_obj["list"] = [0, 1, 2, 3]
js_obj["list"][1] = 1
js_obj["nested_dict"]["key"] = value
js_obj["nested_list_of_dicts"][1]["key"] = 1


### Accessing nested tuples (you can see tuples as immutable lists)

In [14]:
tpl = ("key",({"dict_key":"dict_value"}, 1))
print(f"tpl[1] = {tpl[1]}")
print(f"tpl[1][0] = {tpl[1][0]}")
print(f"tpl[1][0]['dict_key'] = {tpl[1][0]['dict_key']}")

tpl[1] = ({'dict_key': 'dict_value'}, 1)
tpl[1][0] = {'dict_key': 'dict_value'}
tpl[1][0]['dict_key'] = dict_value


### Iterating over keys and values of a dict

In [15]:
js_obj = {
    "a": 1,
    "b": 2,
    "c": 3
}
for key, value in js_obj.items():
    print(key, value)

a 1
b 2
c 3


### List comprehensions and their equivalent in a regular for-loop

In [16]:
list1 = [i for i in range(3)]
list2 = []
for i in range(3):
    list2.append(i)
print(list1)
print(list2)

[0, 1, 2]
[0, 1, 2]


### Dict comprehensions and their equivalent in a regular for-loop

In [17]:
multiplied_dict1 = {k: v*2 for k, v in js_obj.items()}
multiplied_dict2 = {}
for k, v in js_obj.items():
    multiplied_dict2[k] = v*2
print(multiplied_dict1)
print(multiplied_dict2)

{'a': 2, 'b': 4, 'c': 6}
{'a': 2, 'b': 4, 'c': 6}


### Regular expressions: first compiling into a regex object

In [24]:
match_str = "blabla"
nomatch_str = "blbl"
bla_regex = re.compile("(bla)*")
print(bla_regex.search(match_str).groups())
print(re.search("(bla)*",match_str).groups())
print(bla_regex.search(nomatch_str).groups())

('bla',)
('bla',)
(None,)


### Finding all regex matches

In [28]:
print(re.findall("bla","bla132465789bla#bla"))

['bla', 'bla', 'bla']


### Excercise 1: Wordcount
For the first excercise we will be performing wordcount on a sample file.

First, the textfile, located in "data/wordcount.txt" needs to be converted to an RDD, and then split up.
Hint: use sc.textFile and flatMap

In [30]:
words = sc.textFile("data/wordcount.txt").flatMap(lambda line: line.split(" "))
print(words.take(3))

['This', 'is', 'a']


Now, count the actual word. Hint: Use map & reduceByKey

In [31]:
wordCounts=words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
print(wordCounts.take(1))

[('This', 1)]


Finally, save the output in a human readable format. You can check the output of the wordcount by going to "data/output/part-00000". Note, if you re-run the following black, you wil get errors because the output files already exist. If you want to re-run this part of the code, delete the output files first.

In [6]:
wordCounts.saveAsTextFile("data/output/")

## Excercise 2: Ecommerce data
In this integrated excercise, we will be processing ecommerce data
### Let's load the ecommerce data and transform it into an RDD
#### the file contains JSON objects of the form

```
{
  "": "9",
  "Clothing ID": "1077",
  "Age": "34",
  "Title": "Such a fun dress!",
  "Review Text": "I'm 5\"5' and 125 lbs. i ordered the s petite to make sure the length wasn't too long. i
 typically wear an xs regular in retailer dresses. if you're less busty (34b cup or smaller), a s petite w
ill fit you perfectly (snug, but not tight). i love that i could dress it up for a party, or down for work
. i love that the tulle is longer then the fabric underneath.",
  "Rating": "5",
  "Recommended IND": "1",
  "Positive Feedback Count": "0",
  "Division Name": "General",
  "Department Name": "Dresses",
  "Class Name": "Dresses"
}
```

In [32]:
ecommerce_data = sc.textFile("./data/ecommerce.json")
print(ecommerce_data.take(1))

['{"": "0", "Clothing ID": "767", "Age": "33", "Title": "", "Review Text": "Absolutely wonderful - silky and sexy and comfortable", "Rating": "4", "Recommended IND": "1", "Positive Feedback Count": "0", "Division Name": "Initmates", "Department Name": "Intimate", "Class Name": "Intimates"}']


### First convert each line (which is a string) to JSON using json.loads

#### remember: Spark uses lazy evaluation so applying map on an RDD does not apply the function on the data until you call a fucntion like collect(), take(), reduce(), count(), saveAsTextFile()... which triggers the execution

#### Hint: also put the RDD in memory using persist

In [33]:
ecommerce_json = ecommerce_data.map(json.loads).persist()
print(ecommerce_json.take(1))

[{'': '0', 'Clothing ID': '767', 'Age': '33', 'Title': '', 'Review Text': 'Absolutely wonderful - silky and sexy and comfortable', 'Rating': '4', 'Recommended IND': '1', 'Positive Feedback Count': '0', 'Division Name': 'Initmates', 'Department Name': 'Intimate', 'Class Name': 'Intimates'}]


### Let's first try to determine the average age of the reviewers on this e-commerce website

#### First construct an RDD where we extract the Age field of each JSON record and transform it to int

In [34]:
age_rdd = ecommerce_json.map(lambda js: int(js["Age"]))
print(age_rdd.take(1))

[33]


#### Let's make a sum of all ages so that we can later devide by the total number of records to get the average

In [35]:
age_sum = age_rdd.reduce(lambda x, y: x+y)
print(age_sum)

1014561


#### Now we still need to figure out how many records are in the RDD, use the count() function

In [11]:
n_records = age_rdd.count()

#### compute the average now using the sum and the number of records

In [12]:
avg_age = age_sum/n_records

In [13]:
assert np.isclose(avg_age,43.1985438)

### We will now do the same, but compute the average number of words in the Review Text
#### you can simply use the split function with the space character to split the text into words

In [14]:
nwords_sum = ecommerce_json.map(lambda js: len(js["Review Text"].split(" "))).reduce(lambda x, y: x+y)
avg_nwords = nwords_sum / n_records

In [15]:
assert np.isclose(avg_nwords, 58.0843907)

### Next we will count the number of reviews per rating (i.e. 1, 2, 3, 4 and 5)

#### Similar to the word count exercise, build a RDD of tuples with the rating and a count of 1

In [16]:
rating_counts = ecommerce_json.map(lambda js: (js["Rating"],1)).reduceByKey(lambda x, y: x+y).collect()

#### sort the resulting tuples (after collecting the results) from high to low

In [17]:
sorted_rating_counts = sorted(rating_counts, key = lambda x: x[1], reverse = True)

In [18]:
assert sorted_rating_counts[0] == ('5', 13131)

### A second way to achieve the same thing is by first grouping all records together with the same key and then counting how many appear

In [19]:
rating_counts = ecommerce_json.map(lambda js: (js["Rating"], None)).groupByKey().map(lambda x: (x[0],len(x[1])))

In [20]:
sorted_rating_counts = sorted(rating_counts.collect(), key = lambda x: x[1], reverse = True)

In [21]:
assert sorted_rating_counts[0] == ('5', 13131)

### Next we will count the number of reviews per Department Name

In [22]:
category_counts = ecommerce_json.map(lambda js: (js["Department Name"],1)).reduceByKey(lambda x, y: x+y)

#### let's filter out all the Department names with less than 1000 reviews

In [23]:
category_counts_filtered = category_counts.filter(lambda x: x[1]>=1000).collect()

In [24]:
assert len(category_counts_filtered) == 5

#### select the Department with most reviews

In [25]:
largest_category = sorted(category_counts_filtered, key = lambda x: x[1], reverse=True)[0][0]

In [26]:
assert largest_category == "Tops"

#### slightly more complex, let's now count the number of reviews per Department Name and Rating

In [27]:
category_rating_counts = (ecommerce_json
                          .map(lambda js: ((js["Department Name"],js["Rating"]),1))
                          .reduceByKey(lambda x, y: x+y)
                         )

#### only keep the counts for the Jackets department and sort from high to low number of reviews

In [28]:
jackets = category_rating_counts.filter(lambda x: x[0][0]=="Jackets").collect()

In [29]:
sorted_jackets = sorted(jackets, key = lambda x: x[1], reverse=True)

In [30]:
assert sorted_jackets[0] == (("Jackets",'5'), 631)

### Create per Clothing ID a list of all the ages

In [31]:
id2agelist = (ecommerce_json
              .map(lambda js: (js["Clothing ID"], int(js["Age"])))
              .groupByKey()
              .map(lambda x: (x[0],list(x[1])))
             )

#### keep only the Clothing IDs that have more than 500 reviews and compute both the average and standard deviation of the age per Clothing ID

#### The output should be tuples of the following form

```
('Clothing ID', {"avg": 43.4564, "std": 12.14566})
```

Hint: use np.array and np.std and np.mean

In [32]:
def compute_summary_statistics(tpl):
    id, age_list = tpl
    age_array = np.array(age_list)
    return (id, {"avg": np.mean(age_array), "std": np.std(age_array)})

stats = id2agelist.filter(lambda x: len(x[1])>500).map(compute_summary_statistics)

#### sort the results according the average age from high to low

In [33]:
sorted_stats = sorted(stats.collect(), key=lambda x:x[1]["avg"], reverse=True)

In [34]:
assert sorted_stats[0] == ('829', {'avg': 44.64136622390892, 'std': 12.447020037376479})

### Next we will compute the term frequency - inverse document frequency of all the words in the reviews

https://en.wikipedia.org/wiki/Tf%E2%80%93idf <br>

#### First we will compute the document frequencies, i.e. count how many times each word occurs in a review (use a regular expression for this to filter out real words (no punctuation and numbers)

The output of the document_frequencies RDD should be tuples of the form

```
('word', document_count)
```



In [35]:
#define the regular expression to extract only words with at least one character
regex = re.compile("[a-zA-Z]+")

In [36]:
def document_terms(js):
    words = set(regex.findall(js["Review Text"]))
    for word in words:
        yield (word.lower(),1)
      
#use flatMap
document_frequencies = ecommerce_json.flatMap(document_terms).reduceByKey(lambda x, y: x+y)

In [37]:
assert document_frequencies.filter(lambda x: x[0]=="wonderful").collect()[0][1] == 290

#### Now we will compute the inverse document frequency using the formula np.log(number of documents/document count)

In [38]:
inverse_document_frequencies = document_frequencies.map(lambda x: (x[0], np.log(n_records/x[1])))
print(inverse_document_frequencies.take(2))

[('absolutely', 3.3820511804752176), ('sexy', 4.717052247207557)]


In [39]:
assert sorted(inverse_document_frequencies.collect(), key = lambda x: x[1], reverse=True)[0][0] == "narrowing"

### We will now compute the term frequencies per document

#### The output should be an RDD of tuples of the form

The document_id can be found under the empty key, i.e. js[""]

```
('word', ('document_id', count))
```

In [40]:
def document_term_frequencies(js):
    record_id = js[""]
    words = regex.findall(js["Review Text"])
    word_count = Counter()
    for word in words:
        word_count[(record_id, word.lower())] += 1
    return [(word,(record_id,cnt)) for ((record_id, word), cnt) in word_count.items()]

term_frequencies = ecommerce_json.flatMap(document_term_frequencies)
print(term_frequencies.take(2))

[('absolutely', ('0', 1)), ('wonderful', ('0', 1))]


In [41]:
assert term_frequencies.filter(lambda x: x[0]=="it" and x[1][0]=='1').collect()[0][1][1] == 4

### Now we need to join the document frequencies with the term frequencies per document

#### the output of the join will be an RDD with tuples of the form

```
('word', (idf, ('document_id', word count/term frequency) ) )
```

In [42]:
tf_idf_joined = inverse_document_frequencies.join(term_frequencies)
print(tf_idf_joined.take(2))

[('comfortable', (2.073244314833701, ('0', 1))), ('comfortable', (2.073244314833701, ('2', 1)))]


In [43]:
assert tf_idf_joined.filter(lambda x: x[0]=='comfortable' and x[1][1][0]=="0").collect()[0] == ('comfortable', (2.073244314833701, ('0', 1)))

### Now lets multiply the term frequency/word count of each document with the inverse document frequency of the word

#### and now make the keys (i.e. first element of the tuples) of the records in the resulting RDD the id of the document

The output of this RDD should look like:

```
('document_id', ('word', tf*idf))
```

In [44]:
tf_idf = tf_idf_joined.map(lambda x: (x[1][1][0], (x[0], x[1][0] * x[1][1][1])))
print(tf_idf.take(2))

[('0', ('comfortable', 2.073244314833701)), ('2', ('comfortable', 2.073244314833701))]


In [45]:
assert np.isclose(tf_idf.filter(lambda x: x[0]=='0' and x[1][0]=="comfortable").collect()[0][1][1], 2.07324431)

### Finally we need to group all words that belong to the same document

#### The output should be a tuple of the form ('document_id', words_dict) where words_dict is a dict of the following format

```
{
    "word": tf_idf_word
    "word2": tf_idf_word2
    ...
}
```

In [46]:
def review_tf_idf(tpl):
    id, tpls = tpl
    words = {word: tf_idf for word, tf_idf in tpls}
    return (id, words)
        
per_review = tf_idf.groupByKey().map(review_tf_idf)

In [47]:
expected = {'comfortable': 2.073244314833701, 'silky': 5.235846040622725, 'wonderful': 4.394278854944507, 'absolutely': 3.3820511804752176, 'sexy': 4.717052247207557, 'and': 0.36522226581530454}
assert per_review.filter(lambda x: x[0] == '0').collect()[0][1] == expected